In [2]:
import pandas as pd
import re
import string
import numpy as np
from numpy import ndarray, dot, array, float32 as REAL
from myFunctions import processSentences, meanVector, cosineDistance
import plotly.express as px
import statistics
import math

# NLP Packages
from gensim.models.phrases import Phrases, Phraser
from gensim import matutils
from gensim.models import KeyedVectors


/Users/brianward/Desktop/Northeastern/AI/project/aiEnv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Import Model

In [3]:
# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("V2.wordvectors", mmap='r')

FileNotFoundError: [Errno 2] No such file or directory: 'V2.wordvectors'

In [3]:
print('Our vocabulary has ', len(wv), 'words in it.')

Our vocabulary has  37298 words in it.


# Lets Explore

In [4]:
wv.most_similar(positive=['gme', 'amc', 'tsla', 'nok', 'etsy', 'crox', 'appl', 'microsoft', 'facebook', 'home_depot', 'autodesk', 'general_motors'], 
                negative=['hoooooold','boyssss', 'goooooooooo', 'babyyyyy'], topn=500)

[('amazon', 0.81451416015625),
 ('apple', 0.7931689023971558),
 ('nvidia', 0.7883455753326416),
 ('aapl', 0.7810547947883606),
 ('nvda', 0.7748650908470154),
 ('amd', 0.7728115916252136),
 ('sony', 0.767702043056488),
 ('amzn', 0.7664901614189148),
 ('msft', 0.7591730952262878),
 ('roku', 0.756515622138977),
 ('goog', 0.755368173122406),
 ('snap', 0.7471686005592346),
 ('xlnx', 0.7463983297348022),
 ('ebay', 0.7436132431030273),
 ('pinterest', 0.742999255657196),
 ('shopify', 0.7419460415840149),
 ('activision', 0.7359938621520996),
 ('fedex', 0.7330340147018433),
 ('pypl', 0.731174647808075),
 ('twtr', 0.7303600311279297),
 ('walmart', 0.7277743220329285),
 ('uber', 0.7268913984298706),
 ('yhoo', 0.7255257964134216),
 ('fb', 0.725010097026825),
 ('wmt', 0.7234241962432861),
 ('tesla', 0.7234176397323608),
 ('tif', 0.720177948474884),
 ('fitbit', 0.7198118567466736),
 ('broadcom', 0.7195560336112976),
 ('hasbro', 0.7185410857200623),
 ('dropbox', 0.7176898121833801),
 ('gopro', 0.71751

In [3]:
# myVec = meanVector(wv, positive=['gme', 'amc', 'tsla', 'nok', 'etsy', 'crox', 'appl', 'microsoft', 'facebook', 'home_depot',], 
#                        negative=['hoooooold','boyssss', 'goooooooooo', 'babyyyyy'])

myVec = meanVector(wv, positive=['gme', 'amc','game_stop','tsla', 'nok', 'etsy', 'crox', 'appl', 'microsoft', 'facebook', 'netflix', 'dropbox','slack', 'peloton', 'wal_mart', 'lululemon'], 
                 negative=[])

In [4]:
type(myVec)

numpy.ndarray

In [1]:
wv.most_similar(positive=[myVec], topn=500)[110:125]

NameError: name 'wv' is not defined

In [8]:

# def cosineDistance(v1, v2):
#     # matutils.unitvec scales the vectors
#     return dot(matutils.unitvec(v1), matutils.unitvec(v2))
    

## Let's look at the distribution of our vector with all the vectors in the keyed Vectors

In [5]:
sims = []

# NOTE: cosineDistance can be found in myFunctions.py

for v in wv.vectors:
    sims.append(cosineDistance(v, myVec))
    



In [8]:
fig = px.histogram(pd.DataFrame({'value':sims}), x="value")
fig.add_vline(x=0.55, line_width=3, line_dash="dash", line_color="red")
fig.update_xaxes(title_text="Similarity")


fig.show()

# Testing against Ground Truth

### Goal : identify company names and tickers. 

In [19]:
groundTruth = pd.read_csv("data/groundTruth.csv")
groundTruth = groundTruth[['sent','expected']]

In [20]:
# lets just tokenize the results. 

def tokenize(sent):
    if type(sent) == str:
        return sent.strip().split(' ')
    else: 
        return []

    
# apply to our expected column
groundTruth['expected'] = groundTruth['expected'].apply(tokenize)

groundTruth.head()

,sent,expected
0,autists gather and now our hold begins,[]
1,weekend_discussion_thread october,[]
2,united knows i have fomo problems and is explo...,[united]
3,x us steel,[]
4,paying dividends on robinhood when calls you s...,[]


In [24]:
'''
So what do we need to do?
    
    - a 100% would look like:
        if there were none in the results of ground truth. There should be none over the threshold.
    - all items in results should be found.
'''
def test(keyedVectors, myVec, groundTruth, threshold, penalty):
    
    modelResults = []
    scores  = []
    
    for i in range(len(groundTruth)):
        
        myMatches = []
        
        # so we need to look at each word individually.
        # i dont want to keep every value... so lets use a threshold.
        for word in groundTruth['sent'][i].split(' '):
            if word in keyedVectors:
                if cosineDistance(myVec, keyedVectors[word]) > threshold :
                    myMatches.append(word)
        
        modelResults.append(list(set(myMatches)))
        
#         print('type : ', type(groundTruth['result'][i]))
#         print('value : ', groundTruth['result'][i])


        
#         if math.isnan(groundTruth['result'][i]):
#             truthResults = []
#         else:
#             truthResults = groundTruth['result'][i]
            
        truthResults = groundTruth['expected'][i]
        
            
        truthSet = set(truthResults)
        matchSet = set(myMatches)
        # okay so what are we supposed to do here?
#         print('truth set : {}'.format(truthSet))
#         print('match set : {}'.format(matchSet))


        missed = len(truthSet - matchSet)
        over = len(matchSet - truthSet)
        
        myScore = 1.0
    
        if missed > 0:
            myScore = (len(truthSet) - missed) / len(truthSet)
#             print('missed! myscore = ', myScore)
        
        if over > 0:
            myScore = myScore - (penalty*over)
#             print('over! myscore = ', myScore)


        scores.append(myScore)
        
    groundTruth['actual'] = modelResults
    
    return statistics.mean(scores)

In [29]:
test(wv, myVec, groundTruth, 0.6, 0.3)

0.7262333333333333

In [23]:
# groundTruth.loc[groundTruth['myResults'] == []]

sent, expected, actual = [], [], []



for i in range(len(groundTruth)):
    
    if len(groundTruth['actual'][i]) > 0 or type(groundTruth['expected'][i]) == str:
        sent.append(groundTruth['sent'][i])
        expected.append(groundTruth['expected'][i])
        actual.append(groundTruth['actual'][i])

TestResults = pd.DataFrame({'sent':sent, 'expected':expected, 'actual': actual})

TestResults[:50]

,sent,expected,actual
0,masa from softbank yolos on tsla and out jpows...,"[softbank, tsla]",[tsla]
1,intel says nm chips still on track despite rep...,[intel],[intel]
2,hcac canoo for the win trevor sank,"[hcac, canoo]",[canoo]
3,snap is almost a penny_stock bet miranda kerr ...,[snap],[snap]
4,sexy chart twtr,[twtr],[twtr]
5,but reddit told_me facebook is gonna die any d...,[facebook],[facebook]
6,why you should short apple,[apple],[apple]
7,please_tell_me someone here yolo d twtr weekly...,[twtr],[twtr]
8,bull_gang amd,[amd],[amd]
9,started with k last_week current positions fb ...,"[fb, gme]",[fb]
